In [0]:
!ls

drive  sample_data


In [0]:
import os
poster_movies_2 = os.listdir("/content/drive/My Drive/storage2")
print(len(poster_movies_2))

19848


In [0]:
poster_movies = poster_movies_2

In [0]:
len(set(poster_movies))

19848

In [0]:
poster_movies_copy_2 = [i.replace("_"," ").rstrip(".jpg") for i in poster_movies_2]


In [0]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 61kB/s 
     |████████████████████████████████| 491kB 38.2MB/s 
     |████████████████████████████████| 3.2MB 53.3MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn





58892288/58889256 [==============================] - 2s 0us/step




In [0]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/last.csv", engine="python")
df2 = df[df['title'].isin(poster_movies_copy_2)]
df2 = df2.reset_index()


NameError: ignored

In [0]:
len(df2)

17824

In [0]:
imnames=poster_movies
feature_list2=[]
genre_list2=[]
file_order2=[]
print("Starting extracting VGG features for scraped images. This will take time, Please be patient...")
print("Total images = ",len(imnames))
failed_files2=[]
succesful_files2=[]
i=0
for mov in range(len(df2)):
    i+=1
    mov_name=df2.title[mov]
    mov_name=mov_name.replace(':','/')
    poster_name=mov_name.replace(' ','_')+'.jpg'
    if poster_name in imnames:
        img_path="/content/drive/My Drive/storage2/"+poster_name
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            succesful_files2.append(poster_name)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            features = model.predict(x)
            file_order2.append(img_path)
            feature_list2.append(features)
            genre_list2.append(df2.genre_ids[mov])
            if np.max(np.asarray(feature_list2))==0.0:
                print('problematic',i)
            if i%50==0 or i==1:
                print("Working on Image : ",i)
        except:
            failed_files2.append(poster_name)
            continue
        
    else:
        continue
print("Done with all features, please pickle for future use!")

Starting extracting VGG features for scraped images. This will take time, Please be patient...
Total images =  19848
Working on Image :  1
Working on Image :  50
Working on Image :  100
Working on Image :  150
Working on Image :  200
Working on Image :  250
Working on Image :  300
Working on Image :  400
Working on Image :  450
Working on Image :  500
Working on Image :  550
Working on Image :  600
Working on Image :  650
Working on Image :  750
Working on Image :  800
Working on Image :  850
Working on Image :  900
Working on Image :  950
Working on Image :  1000
Working on Image :  1050
Working on Image :  1100
Working on Image :  1150
Working on Image :  1200
Working on Image :  1250
Working on Image :  1350
Working on Image :  1400
Working on Image :  1450
Working on Image :  1500
Working on Image :  1550
Working on Image :  1600
Working on Image :  1650
Working on Image :  1700
Working on Image :  1750
Working on Image :  1800
Working on Image :  1850
Working on Image :  1900
Work

In [0]:
len(succesful_files2)

15386

In [0]:
len(failed_files2)

0

In [0]:
import pickle
list_pickled=(feature_list2,file_order2,failed_files2,succesful_files2,genre_list2)
f=open('posters_new_features2.pckl','wb')
pickle.dump(list_pickled,f)
f.close()
print("Features dumped to pickle file")

Features dumped to pickle file


In [0]:
import pickle
f7=open('/content/drive/My Drive/posters_new_features2.pckl','rb')
list_pickled=pickle.load(f7)
f7.close()

In [0]:
#Pickle is indexed, can get individual info out of them
list_pickled[2]

[]

In [0]:
import numpy as np
(feature_list,files,failed,succesful,genre_list)=list_pickled

(a,b,c,d)=feature_list[0].shape
feature_size=a*b*c*d

np_features=np.zeros((len(feature_list),feature_size))
for i in range(len(feature_list)):
    feat=feature_list[i]
    reshaped_feat=feat.reshape(1,-1)
    np_features[i]=reshaped_feat

X=np_features

from sklearn.preprocessing import MultiLabelBinarizer
mlb=MultiLabelBinarizer()
Y=mlb.fit_transform(genre_list)

Y.shape

(15386, 14)

In [0]:
visual_problem_data=(X,Y)
f8=open('visual_problem_data_clean.pckl','wb')
pickle.dump(visual_problem_data,f8)
f8.close()

In [0]:
f8=open('visual_problem_data_clean.pckl','rb')
visual_features=pickle.load(f8)
f8.close()

In [0]:
(X,Y)=visual_features


In [0]:
X.shape

(15386, 25088)

In [0]:
mask = np.random.rand(len(X)) < 0.8


In [0]:
X_train=X[mask]
X_test=X[~mask]
Y_train=Y[mask]
Y_test=Y[~mask]

In [0]:
X_test.shape


(3056, 25088)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
model_visual = Sequential([
    Dense(1024, input_shape=(25088,)),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(14),
    Activation('sigmoid'),
])
opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

#sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.4, nesterov=False)
model_visual.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
model_visual.fit(X_train, Y_train, epochs=10, batch_size=64,verbose=1)


Epoch 1/10
12330/12330 [==============================] - 59s 5ms/step - loss: 0.9928 - acc: 0.6464
Epoch 2/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.5303 - acc: 0.7245
Epoch 3/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.4097 - acc: 0.7977
Epoch 4/10
12330/12330 [==============================] - 58s 5ms/step - loss: 0.3123 - acc: 0.8535
Epoch 5/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.2326 - acc: 0.8984
Epoch 6/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.1720 - acc: 0.9295
Epoch 7/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.1267 - acc: 0.9505
Epoch 8/10
12330/12330 [==============================] - 58s 5ms/step - loss: 0.0918 - acc: 0.9662
Epoch 9/10
12330/12330 [==============================] - 58s 5ms/step - loss: 0.0670 - acc: 0.9768
Epoch 10/10
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0521 - acc: 0.9826

In [0]:
model_visual.fit(X_train, Y_train, epochs=50, batch_size=64,verbose=1)


Epoch 1/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0396 - acc: 0.9872
Epoch 2/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0310 - acc: 0.9902
Epoch 3/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0243 - acc: 0.9927
Epoch 4/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0196 - acc: 0.9945
Epoch 5/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0162 - acc: 0.9954
Epoch 6/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0148 - acc: 0.9958
Epoch 7/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0109 - acc: 0.9971
Epoch 8/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0107 - acc: 0.9971
Epoch 9/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0095 - acc: 0.9975
Epoch 10/50
12330/12330 [==============================] - 57s 5ms/step - loss: 0.0092 - acc: 0.9977

In [0]:
Y_preds=model_visual.predict(X_test)


In [0]:
sum(sum(Y_preds))


24512.98642730713

In [0]:
f6=open('/content/drive/My Drive/Genredict.pckl','rb')
Genre_ID_to_name=pickle.load(f6)
f6.close()

In [0]:
sum(Y_preds[1])


6.273415446281433

In [0]:
sum(Y_preds[2])


4.000004172325134

In [0]:
genre_list=sorted(list(Genre_ID_to_name.keys()))


In [0]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [0]:
precs=[]
recs=[]
for i in range(len(Y_preds)):
    row=Y_preds[i]
    gt_genres=Y_test[i]
    gt_genre_names=[]
    for j in range(14):
        if gt_genres[j]==1:
            gt_genre_names.append(Genre_ID_to_name[genre_list[j]])
    top_3=np.argsort(row)[-4:]
    predicted_genres=[]
    for genre in top_3:
        predicted_genres.append(Genre_ID_to_name[genre_list[genre]])
    (precision,recall)=precision_recall(gt_genre_names,predicted_genres)
    precs.append(precision)
    recs.append(recall)
    print("Predicted: ",','.join(predicted_genres)," Actual: ",','.join(gt_genre_names))

Predicted:  Drama,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Horror,Action,History,Crime,Science Fiction,Mystery
Predicted:  Horror,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Drama,Horror,Comedy,Thriller,Crime,Science Fiction,Mystery
Predicted:  Horror,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Drama,Horror,Action,History,Thriller,Crime,Science Fiction,Mystery
Predicted:  Horror,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Horror,Action,History,Crime,Science Fiction,Mystery
Predicted:  Adventure,Fantasy,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Drama,Horror,Action,Comedy,History,Thriller,Crime,Science Fiction,Mystery
Predicted:  Drama,Thriller,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Horror,Action,History,Crime,Science Fiction,Mystery
Predicted:  Adventure,Fantasy,Science Fiction,Mystery  Actual:  Adventure,Fantasy,Animation,Drama,Horror,History,Thriller,Crime,Science Fiction,Myst

In [0]:
print(np.mean(np.asarray(precs)),np.mean(np.asarray(recs)))
print("avg=",(np.mean(np.asarray(precs))+np.mean(np.asarray(recs)))/2)

0.8461223821989529 0.48723005197972474
avg= 0.6666762170893388
